In [31]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import os
from copy import deepcopy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from itertools import permutations
import torch.nn.functional as F
# from tqdm import tqdm
# import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import vector
from tqdm.notebook import tqdm
import yaml

In [32]:
yaml_name='Aug16_simple.yaml'

In [33]:
base_path = os.path.dirname(os.getcwd())

full_folder_path = os.path.join(base_path,"saved_files", "fake_data")

yaml_file_path= os.path.join(base_path, "fnn_FeatureRegression", 'yamls', yaml_name)
with open(yaml_file_path, 'r') as yaml_file:
    loaded_data = yaml.load(yaml_file, Loader=yaml.SafeLoader)

# Extract the general configurations
data_name = loaded_data['data_name']
batch_size = loaded_data['batch_size']
early_stop_patience = loaded_data['early_stop_patience']
learning_rate = loaded_data['learning_rate']
Local_model=loaded_data['Local_model']
model_save_prefix=loaded_data['model_save_prefix']

# Extracting model configurations
models = loaded_data['models']
hidden_layers_list_loaded = [model['hidden_layers'] for model in models]
activation_fn_names_loaded = [model['activation_fn'] for model in models]

# Map function names back to their actual functions
activation_fn_mapping = {
    'relu': F.relu,
    'sigmoid': F.sigmoid,
    'tanh': F.tanh
}

# Convert function names to actual functions
activation_fn_list_loaded = [activation_fn_mapping[name] for name in activation_fn_names_loaded]


In [34]:
base_path = os.path.dirname(os.getcwd())

full_folder_path = os.path.join(base_path,"saved_files", "fake_data")
# data_df=pd.read_pickle(os.path.join(full_folder_path,"Aug7_1mil.pkl"))
# with open(os.path.join(full_folder_path,"Aug10_5mil.pkl"), 'rb') as f:
with open(os.path.join(full_folder_path,data_name), 'rb') as f:
    clean_data_dict = pickle.load(f)
print(clean_data_dict.keys())
numevents=len(clean_data_dict['2_phi'])
print("number of events:",numevents)

data_dict=clean_data_dict
print(data_dict.keys())

data_dict_np={}
for key in data_dict.keys():
    data_dict_np[key]=np.array(data_dict[key])

yaml_file_path= os.path.join(base_path, "fnn_FeatureRegression", 'yamls', yaml_name)


dict_keys(['event', 'genWeight', 'MET_phi', '1_phi', '1_genPartFlav', '2_phi', '2_genPartFlav', '3_phi', '3_genPartFlav', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'ma

In [37]:
print("max of pt 1,2,3:", np.max(data_dict['pt_1']), np.max(data_dict['pt_2']), np.max(data_dict['pt_3']))
print("min of pt 1,2,3:", np.min(data_dict['pt_1']), np.min(data_dict['pt_2']), np.min(data_dict['pt_3']))

max of pt 1,2,3: 451.1299035814137 539.8878152156531 469.05998514965927
min of pt 1,2,3: 19.453018641861117 20.86271328417876 13.620068613240143


In [5]:
input_data_names_ordered = [
    ['MET_phi', 'pt_MET'], 
    ['1_phi', 'pt_1', 'eta_1', 'mass_1'], 
    ['2_phi', 'pt_2', 'eta_2', 'mass_2'], 
    ['3_phi', 'pt_3', 'eta_3', 'mass_3']
]
input_data_particle_order = ['MET', '1', '2', '3']

pair_order = ["MET_1", "MET_2", "MET_3", "1_2", "1_3", "2_3"]
used_labels2 = [
    ['deltaphi_1MET', 'mt_1MET'], 
    ['deltaphi_2MET', 'mt_2MET'], 
    ['deltaphi_3MET', 'mt_3MET'], 
    ['deltaphi_12', 'deltaeta_12'], 
    ['deltaphi_13', 'deltaeta_13'], 
    [ 'deltaphi_23', 'deltaeta_23']
]

lepton_input_ordered = input_data_names_ordered[1:]
lepton_output_ordered = used_labels2[3:]

l_input_shape=(numevents,len(lepton_input_ordered), len(lepton_input_ordered[0]))
print("events, particles, input features: ",l_input_shape)
l_input= np.empty(l_input_shape)

for i in range(len(lepton_input_ordered)):
    for j, feature in enumerate(lepton_input_ordered[i]):
        l_input[:,i,j] = data_dict_np[feature]

l_output_shape=(numevents, len(lepton_output_ordered), len(lepton_output_ordered[0]))
print("events, particle pairs, output kin. features: ",l_output_shape)
l_output= np.empty(l_output_shape)

for i in range(len(lepton_output_ordered)):
    for j, feature in enumerate(lepton_output_ordered[i]):
        l_output[:,i,j] = data_dict_np[feature]

lepton_pair_order = pair_order[3:]
lepton_particle_order = input_data_particle_order[1:]
print("lepton pair order: ", lepton_pair_order)
print("lepton particle order: ", lepton_particle_order)

events, particles, input features:  (2844237, 3, 4)
events, particle pairs, output kin. features:  (2844237, 3, 2)
lepton pair order:  ['1_2', '1_3', '2_3']
lepton particle order:  ['1', '2', '3']


In [8]:
def add_extra_features(data):
    p1_pt=data['pt_1']
    p2_pt=data['pt_2']
    p3_pt=data['pt_3']

    p1_phi=data["1_phi"]
    p2_phi=data["2_phi"]
    p3_phi=data["3_phi"]

    p1_eta=data["eta_1"]
    p2_eta=data["eta_2"]
    p3_eta=data["eta_3"]

    p1_mass=data["mass_1"]
    p2_mass=data["mass_2"]
    p3_mass=data["mass_3"]

    particle1=vector.arr({"pt": p1_pt, "phi": p1_phi, "eta": p1_eta, "mass": p1_mass})
    particle2=vector.arr({"pt": p2_pt, "phi": p2_phi, "eta": p2_eta, "mass": p2_mass})
    particle3=vector.arr({"pt": p3_pt, "phi": p3_phi, "eta": p3_eta, "mass": p3_mass})

    p4_mother12=particle1+particle2
    p4_mother23=particle2+particle3
    p4_mother13=particle1+particle3

    pairs=['12','13','23']
    motherpairs=[p4_mother12, p4_mother13, p4_mother23]
    # features_toadd=[ 'mass', 'pt', 'eta' , 'phi',  'px', 'py', 'pz', 'energy']
    features_toadd=[ 'mass', 'pt', 'eta']

    add_feat_size=(len(data['pt_1']), len(pairs), len(features_toadd))
    add_feat_array= np.empty(add_feat_size)

    for feature in features_toadd:
        for i, pair in enumerate(pairs):
           add_feat_array[:, i, features_toadd.index(feature)] = getattr(motherpairs[i], feature)
    return add_feat_array

data_conc=add_extra_features(data_dict_np)
print(data_conc.shape)
# print(data_new.columns)
l_output2= np.concatenate((l_output, data_conc), axis=2)
print("l_output new shape: ",l_output2.shape)
l_output2_shape=l_output2.shape


(2844237, 3, 3)
l_output new shape:  (2844237, 3, 5)


In [17]:
print(l_input.shape)
# pair_input_order=[(0,1),(0,2),(1,2), (1,0),(2,0),(2,1)]
pair_input_order=[(0,1)]



output_dim=3*l_output2_shape[2]
input_dim=l_input.shape[2]*2
# datashape=(numevents*len(pair_input_order),l_input.shape[2]*2)
datashape=(numevents,input_dim)
print("datashape",datashape)
data=np.array(np.zeros(datashape))
for i in range(len(pair_input_order)):
    combined=np.concatenate((l_input[:,pair_input_order[i][0],:],l_input[:,pair_input_order[i][1],:]),axis=1)
    # print(combined.shape)
    #add to data
    data[:,i*combined.shape[1]:(i+1)*combined.shape[1]]=combined



print("data shape",data.shape)
labels=l_output2.reshape((numevents,output_dim))
print("labels shape",labels.shape)

(2844237, 3, 4)
datashape (2844237, 8)
data shape (2844237, 8)
labels shape (2844237, 15)


In [29]:
pair_input_order=[(0,1),(0,2),(1,2), (1,0),(2,0),(2,1)]
print("original input shape:", l_input.shape)
print("original output shape:", l_output2.shape)

pairdatashape=(numevents,len(pair_input_order), int(l_input.shape[2]*2))
pairdata=np.empty(pairdatashape)
# print("pairdatashape:", pairdatashape)
for i, pair in enumerate(pair_input_order):
    pairdata[:,i,:l_input.shape[2]] = l_input[:,pair[0],:]
    pairdata[:,i,l_input.shape[2]:] = l_input[:,pair[1],:]
print("pairdata shape:", pairdata.shape)
data=pairdata.reshape(numevents*len(pair_input_order), int(pairdata.shape[2]))
print("input shape:", input.shape)

output_half=l_output2.reshape(numevents*3, int(l_output2.shape[2]))
print("output_half shape:", output_half.shape)
labels=np.vstack((output_half, output_half))
print("labels shape:", labels.shape)


original input shape: (2844237, 3, 4)
original output shape: (2844237, 3, 5)
pairdata shape: (2844237, 6, 8)
input shape: (17065422, 8)
output_half shape: (8532711, 5)
labels shape: (17065422, 5)


In [30]:
print("output_dim = ", int(3*l_output2_shape[2]))
print("input dim", int(l_input.shape[2]*6*2))

output_dim =  15
input dim 48


In [21]:

print("l_output2 shape: ", l_output2.shape)
output_smaller=l_output2[:,0, :]
print("smaller output shape: ", output_smaller.shape)
output_dim=l_output2_shape[2]
print("numevents: ", numevents)
labels=output_smaller.reshape((numevents,output_dim))
print("labels shape",labels.shape)

l_output2 shape:  (2844237, 3, 5)
smaller output shape:  (2844237, 5)
numevents:  2844237
labels shape (2844237, 5)


In [8]:
batch = 320
x_tensor = torch.from_numpy(data).float()
y_tensor = torch.from_numpy(labels).float()

# dataset=particleDataset(data,y)
dataset=TensorDataset(x_tensor,y_tensor)

train_val_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
training_data, val_data = train_test_split(train_val_data, test_size=0.2, random_state=42)

train_loader=DataLoader(training_data, batch_size=batch, shuffle=True)
test_loader=DataLoader(test_data, batch_size=batch, shuffle=False)
full_batch_size = len(val_data)
val_loader = DataLoader(val_data, batch_size=full_batch_size, shuffle=False)


# train_loader=DataLoader(dataset, batch_size=batch, shuffle=True)
#make trainloader using data as x and y as labels
# trainloader=DataLoader(data, batch_size=batch, shuffle=True)
# train_dataset= 
# trainloader=DataLoader(data, batch_size=batch, shuffle=True)

In [9]:
print(len(val_loader))

1


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class KinematicNet(nn.Module):
    def __init__(self):
        super(KinematicNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, int(input_dim*1.5))
        self.fc2 = nn.Linear(int(input_dim*1.5), int(input_dim//3))
        self.fc3 = nn.Linear(int(input_dim//3), output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


class CustomKinematicNet(nn.Module):
    def __init__(self, input_size, hidden_layers, lenoutput, activation_fn=F.relu):
        """
        Args:
        - input_size (int): Size of the input layer.
        - hidden_layers (list of int): Sizes of each hidden layer.
        - lenoutput (int): Size of the output layer.
        - activation_fn (callable): Activation function to use.
        """
        super(CustomKinematicNet, self).__init__()
        
        # Create the list of layers
        layers = [nn.Linear(input_size, hidden_layers[0])]
        for i in range(len(hidden_layers) - 1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
        layers.append(nn.Linear(hidden_layers[-1], lenoutput))
        
        self.layers = nn.ModuleList(layers)
        self.activation_fn = activation_fn
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation_fn(layer(x))
        return self.layers[-1](x)
    




def custom_train_loss(y_pred, y_true):
    mse_loss = torch.mean((y_pred - y_true) ** 2)

    return mse_loss
    
    

def custom_val_loss(y_pred, y_true):
    num_features = int(output_dim / 3)
    y_pred_reshaped = y_pred.reshape(-1, 3, num_features)
    y_true_reshaped = y_true.reshape(-1, 3, num_features)
    
    loss_list = []
    
    for i in range(num_features):
        y_p = y_pred_reshaped[:, :, i].flatten()
        y_t = y_true_reshaped[:, :, i].flatten()
        loss = torch.mean((y_p - y_t) ** 2)
        loss_list.append(loss.item())

    mse_loss = torch.mean((y_pred - y_true) ** 2)
    
    return loss_list, mse_loss

        
        
        
def l2_regularization(model, lambda_reg):
    l2_reg = 0.0
    for W in model.parameters():
        l2_reg += torch.sum(W ** 2)
    return l2_reg * lambda_reg      
        



In [11]:
# (self, input_size, hidden_layers, lenoutput

hidden_layers=[60,80,70,60,50,40,30,20,20]

model = CustomKinematicNet(input_size=input_dim, hidden_layers=hidden_layers, lenoutput=output_dim)
model.to(device)

CustomKinematicNet(
  (layers): ModuleList(
    (0): Linear(in_features=48, out_features=60, bias=True)
    (1): Linear(in_features=60, out_features=80, bias=True)
    (2): Linear(in_features=80, out_features=70, bias=True)
    (3): Linear(in_features=70, out_features=60, bias=True)
    (4): Linear(in_features=60, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=40, bias=True)
    (6): Linear(in_features=40, out_features=30, bias=True)
    (7): Linear(in_features=30, out_features=20, bias=True)
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Linear(in_features=20, out_features=15, bias=True)
  )
)

In [12]:



optimizer=torch.optim.Adam(model.parameters(), lr=0.0001)
# loss_fn=nn.MSELoss()


out_feats=['deltaphi', 'deltaeta', 'mass', 'pt', 'eta']
losses_cols=['train_loss', 'val_loss', 'l2sum']+out_feats
losses_df=pd.DataFrame(columns=losses_cols)


numepochs=10000
best_loss=np.inf
for epoch in range(numepochs):
    model.train()
    train_loss=0
    l2sum=0
    for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader), leave=False, position=0, disable=True):
        x=x.to(device)
        y=y.to(device)
        y_pred=model(x)
        original_loss = custom_train_loss(y_pred, y)
        l2_loss = l2_regularization(model, lambda_reg=1e-7)
        loss = original_loss + l2_loss
        l2sum+=l2_loss.item()
        train_loss += original_loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    model.eval()
    # patience=20
    with torch.no_grad():
        x,y=next(iter(val_loader))
        # x=val_loader
        x=x.to(device)
        y=y.to(device)
        y_pred=model(x)
        feats_loss, valloss = custom_val_loss(y_pred, y)
        # valloss=sum(feats_loss)/len(feats_loss)
        

        if valloss<best_loss:
            best_loss=valloss
            patience=20
            modelsave=deepcopy(model.state_dict())
        else:
            patience-=1
            if patience==0:
                print('early stopping')
                break
    valloss2=valloss.cpu().float()
    loss_strings = [f"{out_feats[i]}: {feats_loss[i]:.4e}" for i in range(len(out_feats))]
    loss_summary = ", ".join(loss_strings)
    loss_values = [train_loss/len(train_loader), valloss2, l2sum]
    loss_values.extend(feats_loss)

    losses_df.loc[epoch] = loss_values
    # losses_df.loc[epoch]=[train_loss/len(train_loader), valloss2, feats_loss[0], feats_loss[1]]
    print(f"epoch: {epoch}, train: {train_loss/len(train_loader):.4e}, val: {valloss2:.4e}, {loss_summary}")


epoch: 0, train: 1.4443e+03, val: 1.0565e+03, deltaphi: 1.5167e+00, deltaeta: 1.5656e+00, mass: 4.0702e+03, pt: 1.2064e+03, eta: 2.8401e+00
epoch: 1, train: 1.0243e+03, val: 1.0073e+03, deltaphi: 1.1743e+00, deltaeta: 9.2941e-01, mass: 3.9587e+03, pt: 1.0728e+03, eta: 2.8202e+00
epoch: 2, train: 9.9455e+02, val: 9.8674e+02, deltaphi: 1.5969e+00, deltaeta: 1.0802e+00, mass: 3.8828e+03, pt: 1.0449e+03, eta: 3.3237e+00
epoch: 3, train: 9.7412e+02, val: 9.6217e+02, deltaphi: 1.5730e+00, deltaeta: 9.8077e-01, mass: 3.7793e+03, pt: 1.0263e+03, eta: 2.6652e+00
epoch: 4, train: 9.4262e+02, val: 9.2559e+02, deltaphi: 9.2666e-01, deltaeta: 7.7831e-01, mass: 3.5868e+03, pt: 1.0370e+03, eta: 2.4535e+00
epoch: 5, train: 8.9779e+02, val: 8.6416e+02, deltaphi: 9.0081e-01, deltaeta: 7.5763e-01, mass: 3.2324e+03, pt: 1.0843e+03, eta: 2.4323e+00
epoch: 6, train: 8.3216e+02, val: 8.0841e+02, deltaphi: 9.0380e-01, deltaeta: 7.3221e-01, mass: 2.8484e+03, pt: 1.1895e+03, eta: 2.4889e+00
epoch: 7, train: 7.8

In [1]:
# plotting histogram
# ['deltaphi_12', 'deltaeta_12'],
out_feats=['deltaphi', 'deltaeta', 'mass', 'pt', 'eta']
modeltest = CustomKinematicNet(input_size=input_dim, hidden_layers=hidden_layers, lenoutput=output_dim)
modeltest.load_state_dict(modelsave)
modeltest.to(device)

residuals=np.array([])
y_total=np.array([])
y_pred_total=np.array([])
for i, (x,y) in enumerate(test_loader):
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    y_pred_total=np.append(y_pred_total, y_pred.cpu().detach().numpy())
    y_total=np.append(y_total, y.cpu().detach().numpy())


    # residuals=np.append(residuals, y_pred.cpu().detach().numpy() - y.cpu().detach().numpy())

    # residuals.append(y_pred.cpu().detach().numpy() - y.cpu().detach().numpy())


numfeatures=5
y_pred_total = y_pred_total.reshape(-1,3,numfeatures)
y_total = y_total.reshape(-1,3,numfeatures)

# print(y_pred_total.shape)
# print(y_total.shape)

residuals = [[] for _ in range(numfeatures)]
label_values = [[] for _ in range(numfeatures)]

for i in range(numfeatures):
    y_curr=y_total[:,:,i]
    # print("ycurr shape before reshape", y_curr.shape)
    y_curr=y_curr.reshape(568848*3,1)
    # print("ycurr shape after reshape", y_curr.shape)
    y_pred_curr=y_pred_total[:,:,i]
    y_pred_curr=y_pred_curr.reshape(568848*3,1)
    residuals_curr = y_pred_curr - y_curr
    residuals[i]=residuals_curr
    label_values[i]=y_curr

residuals = [np.array(res_list) for res_list in residuals]  # Convert lists of arrays to arrays
# residual_medians = [np.median(res) for res in residuals]
residual_std_devs = [np.std(res) for res in residuals]
residual_means = [np.mean(res) for res in residuals]


num_rows = 2
num_cols = 3


fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 15))
flat_axes = axes.flatten()

for i, ax in enumerate(flat_axes[:numfeatures]):
    ax.hist(residuals[i], bins=100, edgecolor='k', alpha=0.65)
    ax.axvline(x=residual_means[i] + residual_std_devs[i], color='r', linestyle='--', label=f'+1 std = {residual_means[i] + residual_std_devs[i]:.2f})')
    ax.axvline(x=residual_means[i] - residual_std_devs[i], color='b', linestyle='--', label=f'-1 std = {residual_means[i] - residual_std_devs[i]:.2f})')
    ax.set_title(f'Residuals for {out_feats[i]}')
    ax.set_yscale('log')
    ax.set_xlabel('Residual Value')
    ax.set_ylabel('Frequency')
    ax.legend()
    
    # Display the mean value on the plot
    mean_text = f"Mean: {residual_means[i]:.2f}, std: {residual_std_devs[i]:.5f}"
    ax.text(0.6, 0.85, mean_text, transform=ax.transAxes)
    

for ax in flat_axes[numfeatures:]:
    ax.axis('off')

plt.tight_layout()
plt.show()
    
    

NameError: name 'CustomKinematicNet' is not defined

In [ ]:
for col in losses_df.columns:
    plt.plot(losses_df[col], label=col)
plt.legend()
plt.yscale('log')
plt.show()

NameError: name 'losses_df' is not defined